<a href="https://colab.research.google.com/github/Guo-bot-1998/Appendicitis/blob/master/Others.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3D U-Net

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class Custom3DNet(nn.Module):
    def __init__(self):
        super(Custom3DNet, self).__init__()
        # Define 3D Convolutional layers
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=16, kernel_size=3)
        self.conv2 = nn.Conv3d(in_channels=16, out_channels=16, kernel_size=3)
        self.conv3 = nn.Conv3d(in_channels=16, out_channels=32, kernel_size=3)
        self.conv4 = nn.Conv3d(in_channels=32, out_channels=32, kernel_size=3)
        self.conv5 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv6 = nn.Conv3d(in_channels=64, out_channels=64, kernel_size=3)

        # Define Pooling layer
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)

        # Define Fully Connected layers
        self.fc1 = nn.Linear(in_features=64, out_features=256)  # Adjust in_features according to the output of the last Conv3D layer
        self.fc2 = nn.Linear(in_features=256, out_features=2)

    def forward(self, x):
        # Apply Convolutional layers and Pooling layers
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))
        x = F.relu(self.conv5(x))
        x = self.pool(F.relu(self.conv6(x)))

        # Flatten the output for the Fully Connected layer
        x = torch.flatten(x, 1)

        # Apply Fully Connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        # Apply Softmax
        output = F.softmax(x, dim=1)
        return output

In [ ]:
net = Custom3DNet()
# Cross Entropy Loss
criterion = nn.CrossEntropyLoss()

# Adam Optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001)  # You can adjust the learning rate as needed

num_epochs=100

for epoch in range(num_epochs):  # num_epochs is the number of epochs you want to train for
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):  # trainloader is your data loader for training data
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = net(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # Print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


# 驗證validation set的一個有闌尾炎的scan會連續出現多張異常cut

In [6]:
import pandas as pd

with open("TrainValid_ground_truth.csv", 'r') as f:
  df = pd.read_csv(f)

labels = df['label']
ids = df['id']

FileNotFoundError: ignored

In [ ]:
for id in ids[:10]:
  value = id.split('_')[-1]
  if (len(value)) == 1:
    print(value)

0
1
2
3
4
5
6
7
8


In [3]:
from google.colab import drive
drive.mount('/content/drive/',)

Mounted at /content/drive/


In [12]:
labelpath = '/content/drive/MyDrive/AOCR2024/TrainValid_ground_truth.csv'
def read_label(excel_path) -> pd.DataFrame:
  """Reads a csv file containing ground-truth.
    The csv file should have two columns: 'id' and 'label'.
  """
  with open(excel_path, 'r') as f:
    df = pd.read_csv(f)
    # df.set_index('id', inplace=True)
    return df

df = read_label(labelpath)
l = df['label'].tolist()

current_id = ""
current_label = 0
ones = []
nscan = 0
ntagget = 0




for i in range(len(df)):
  data = df.iloc[i]
  label = data['label']
  id = data['id']
  id_value  = id.split('_')[-1]
  if (len(id_value) <= 3):
      ones.append(int(label))
  else:
    if not cont(l):
      print(f"{current_id} false cont")
      break
    if sum(ones) == 20:
      print(current_id)
      if (int(current_label) == 0):
        print(f"{current_id} false label")
      ntagget +=1
    nscan += 1
    current_id = id
    current_label = label
    ones = []
print(nscan)
print(ntagget)

 false cont
0
0


In [10]:
def cont(l):
  f1 = False
  f2 = False
  for i in range(len(l)):
    if (l[i] == 1):
      f1 = True
    if (l[i] == 0):
      if (f1):
        f2 = True
    if (l[i] == 1) and f2:
      return False
  return True